# SETUP

In [ ]:
!pip install torch einops numpy timm==0.6.13 scipy gcsfs cdsapi xarray zarr netcdf4 matplotlib

In [ ]:
!cd /workspace/aurora_229s
!git pull

In [ ]:
import importlib
from pathlib import Path
import datetime
import numpy as np
import torch
import gc

In [ ]:
from aurora import inference_helper
from aurora.model import aurora, swin3d

def reload():
    importlib.reload(inference_helper)
    importlib.reload(aurora)
    importlib.reload(swin3d)

In [ ]:
def gpu_mem(msg):
    print(f'{msg}:')
    print("\ttorch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("\ttorch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("\ttorch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    print()

def print_timestamp():
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(formatted_time)

In [ ]:
# Variable names
surf_vars_names = [
    ('2t', '2m_temperature'),
    ('10u', '10m_u_component_of_wind'),
    ('10v', '10m_v_component_of_wind'),
    ('msl', 'mean_sea_level_pressure'),
]
static_vars_names = [
    ('z', 'z'),
    ('slt', 'slt'),
    ('lsm', 'lsm')
]
atmos_vars_names = [
    ('t', 'temperature'),
    ('u', 'u_component_of_wind'),
    ('v', 'v_component_of_wind'),
    ('q', 'specific_humidity'),
    ('z', 'geopotential')
]

all_vars_names = surf_vars_names + atmos_vars_names

In [ ]:
model = aurora.Aurora()
model.load_checkpoint("microsoft/aurora", "aurora-0.25-finetuned.ckpt")
# model.load_checkpoint_local(
#     "/workspace/models/hf_ckpt/aurora-0.25-finetuned.ckpt"
# )
model.configure_activation_checkpointing()

# Inference loop

In [ ]:
# Save May for testing
# base_date_list = ["2022-02-01", "2022-04-01", "2022-05-01", "2022-07-01", "2022-11-01"]
base_date_list = ["2022-02-01", "2022-04-01", "2022-07-01", "2022-11-01"]

base_save_dir = Path("/workspace/models/fisher")
base_save_dir.mkdir(exist_ok=True, parents=True)
device = 'cuda'

model.train()
model = model.to(device)

# MAJOR LOOP -- SURF VARS
for sh_var,lh_var in all_vars_names:
    print(sh_var, lh_var)
    print_timestamp()
    print('\n')

    cnt = 0
    mae_losses = []
    grads = {}
    batcher = inference_helper.InferenceBatcher(
        base_date_list=base_date_list,
        data_path=Path("/workspace/data"),
        max_n_days=14,
    )

    while True:
        model.zero_grad() # Critical to zero-out gradients
        batch, labels = batcher.get_batch()
        if batch is None or labels is None:
            break
        print(batcher.day, batcher.time_idx - 1)

        p = next(model.parameters())
        batch = batch.type(p.dtype)
        batch = batch.crop(model.patch_size)
        batch = batch.to(p.device)

        labels = labels.type(p.dtype)
        labels = labels.crop(model.patch_size)

        # preds = model.forward(batch)
        preds = torch.utils.checkpoint.checkpoint(model.forward, batch, use_reentrant=False)

        if (sh_var,lh_var) in surf_vars_names:
            task_pred = preds.surf_vars[sh_var][0, 0]
            ref = labels.surf_vars[sh_var][0,0].to(device)
        else:
            task_pred = preds.atmos_vars[sh_var][0, 0]
            ref = labels.atmos_vars[sh_var][0,0].to(device)

        # Paper uses mean absolute error
        loss = torch.mean(torch.abs(task_pred - ref))
        # loss = torch.utils.checkpoint.checkpoint(loss_fn, ref, task_pred, use_reentrant=False)
        loss.backward()

        for name,param in model.named_parameters():
            if cnt == 0:
                grads[name] = torch.square(param.grad.clone().to('cpu'))
            else:
                grads[name] += torch.swaure(param.grad.clone().to('cpu'))

        cnt += 1
        mae_losses.append(loss.clone().detach().to('cpu').numpy())
        del preds, task_pred, ref, batch, labels, loss
        gc.collect()
        torch.cuda.empty_cache()

    # finished with loop
    task_dir = base_save_dir / lh_var
    task_dir.mkdir(parents=True, exist_ok=True)

    for name,param in grads.items():
        torch.save(param / float(cnt), task_dir / f'{name}.pt')
    np.save(task_dir / f'LOSSES.npy', np.array(mae_losses))

In [ ]:
from pathlib import Path
import datetime
import numpy as np
import torch
import gc

# Save May and August for testing
base_date_list = ["2022-02-01", "2022-04-01", "2022-07-01", "2022-11-01"]

base_save_dir = Path("/workspace/models/fisher")
base_save_dir.mkdir(exist_ok=True, parents=True)
# device = xm.xla_device()
device = 'cuda'

bdl_model = inference_helper.BackboneDecoderLayers(model.backbone.decoder_layers, model.backbone.num_decoder_layers)
bdl_model = bdl_model.to(device)
decoder = model.decoder.to(device)
decoder.eval()

# MAJOR LOOP -- SURF VARS
for sh_var,lh_var in all_vars_names:
    print(sh_var, lh_var)
    print_timestamp()
    print('\n')

    cnt = 0
    mae_losses = []
    grads = {}
    batcher = inference_helper.InferenceBatcher(base_date_list=base_date_list, data_path=download_path)

    while True:
        model.zero_grad() # Critical to zero-out gradients
        batch, labels = batcher.get_batch()
        if batch is None or labels is None:
            break
        print(batcher.day, batcher.time_idx - 1)

        rollout_step = batch.metadata.rollout_step
        batch = inference_helper.preprocess_batch(model=model, batch=batch, device=device)
        torch.cuda.empty_cache()

        p = next(model.parameters())
        labels = labels.type(p.dtype)
        labels = labels.crop(model.patch_size)

        # -------------------------------------
        # 2. Encoder Forward
        with torch.no_grad():
            x, patch_res = inference_helper.encoder_forward(model=model, batch=batch, device=device)
        batch = batch.to('cpu')
        torch.cuda.empty_cache()
        # gpu_mem('ENCODER FWD PASS')

        # 3. Backbone encoder layers forward
        with torch.no_grad():
            x, skips, c, all_enc_res, padded_outs = inference_helper.backbone_encoder_layers_forward(
                model=model, x=x, patch_res=patch_res, rollout_step=rollout_step, device=device,
            )
        torch.cuda.empty_cache()
        # gpu_mem('BACKBONE ENCODER FWD PASS')

        # 6. BDL forward pass
        x = bdl_model.forward(
            x=x, skips=skips, c=c, all_enc_res=all_enc_res, padded_outs=padded_outs, rollout_step=rollout_step,
        )
        del skips, c, all_enc_res
        torch.cuda.empty_cache()
        # gpu_mem('BDL forward pass')

        # 7. Decoder forward pass
        preds = inference_helper.decoder_forward(
            decoder=decoder, x=x, batch=batch, patch_res=patch_res, surf_stats=model.surf_stats,
        )
        del x, batch, patch_res

        # -------------------------------------


        if (sh_var,lh_var) in surf_vars_names:
            task_pred = preds.surf_vars[sh_var][0, 0]
            ref = labels.surf_vars[sh_var][0,0].to(device)
        else:
            task_pred = preds.atmos_vars[sh_var][0, 0]
            ref = labels.atmos_vars[sh_var][0,0].to(device)

        # Paper uses mean absolute error
        loss = torch.mean(torch.abs(task_pred - ref))
        # loss = torch.utils.checkpoint.checkpoint(loss_fn, ref, task_pred, use_reentrant=False)
        loss.backward()

        for name,param in model.named_parameters():
            if cnt == 0:
            grads[name] = torch.square(param.grad.clone().to('cpu'))
            else:
            grads[name] += torch.swaure(param.grad.clone().to('cpu'))

        cnt += 1
        mae_losses.append(loss.clone().detach().to('cpu').numpy())
        del preds, task_pred, ref, batch, labels, skips, c, all_enc_res, padded_outs, patch_res
        gc.collect()
        torch.cuda.empty_cache()

    # finished with loop
    task_dir = base_save_dir / lh_var
    task_dir.mkdir(parents=True, exist_ok=True)

    for name,param in grads.items():
    torch.save(param / float(cnt), task_dir / f'{name}.pt')
    np.save(task_dir / f'LOSSES.npy', np.array(mae_losses))

In [ ]:
from pathlib import Path
import datetime
import numpy as np
import torch
import gc
from aurora.download_data import download_for_day

# Save May and August for testing
base_date_list = ["2022-02-01", "2022-04-01", "2022-07-01", "2022-11-01"]
base_date_list = ["2022-02-01", "2022-04-01"]#, "2022-07-01", "2022-11-01"]

base_save_dir = Path("/workspace/models/fisher")
base_save_dir.mkdir(exist_ok=True, parents=True)
# device = xm.xla_device()
device = 'cuda'

bel_model = inference_helper.BackboneEncoderLayers(model.backbone.encoder_layers)
bel_model = bel_model.to(device)
bdl_model = inference_helper.BackboneDecoderLayers(model.backbone.decoder_layers, model.backbone.num_decoder_layers)
bdl_model = bdl_model.to(device)
decoder = model.decoder.to(device)
decoder.eval()

# MAJOR LOOP -- SURF VARS
for sh_var,lh_var in all_vars_names:
    print(sh_var, lh_var)
    print_timestamp()
    print('\n')

    cnt = 0
    mae_losses = []
    grads = {'backbone_encoder':{}, 'backbone_decoder':[]}
    download_for_day(day=base_date_list[0], download_path=Path("/workspace/data"))
    batcher = inference_helper.InferenceBatcher(
        base_date_list=base_date_list,
        data_path=Path("/workspace/data"),
        max_n_days=3
    )

    while True:
        model.zero_grad() # Critical to zero-out gradients
        try:
            batch, labels = batcher.get_batch()
        except:
            break
        if batch is None or labels is None:
            break
        print(batcher.day, batcher.time_idx - 1)

        rollout_step = batch.metadata.rollout_step
        batch = inference_helper.preprocess_batch(model=model, batch=batch, device=device)
        torch.cuda.empty_cache()

        p = next(model.parameters())
        labels = labels.type(p.dtype)
        labels = labels.crop(model.patch_size)

        # -------------------------------------
        # 2. Encoder Forward
        with torch.no_grad():
            x, patch_res = inference_helper.encoder_forward(model=model, batch=batch, device=device)
        batch = batch.to('cpu')
        torch.cuda.empty_cache()
        # gpu_mem('ENCODER FWD PASS')

        # 3. Backbone encoder layers forward
        with torch.no_grad():
            c, all_enc_res, padded_outs = inference_helper.backbone_prep(model=model, x=x, patch_res=patch_res, device=device)
            # x, skips, c, all_enc_res, padded_outs = inference_helper.backbone_encoder_layers_forward(
            #     model=model, x=x, patch_res=patch_res, rollout_step=rollout_step, device=device,
            # )
        torch.cuda.empty_cache()
        # gpu_mem('BACKBONE ENCODER FWD PASS')

        # Step 4?
        x, skips = bel_model.forward(x=x, c=c, all_enc_res=all_enc_res, rollout_step=rollout_step)
        torch.cuda.empty_cache()

        # 6. BDL forward pass
        x = bdl_model.forward(
            x=x, skips=skips, c=c, all_enc_res=all_enc_res, padded_outs=padded_outs, rollout_step=rollout_step,
        )
        del skips, c, all_enc_res, padded_outs
        torch.cuda.empty_cache()
        # gpu_mem('BDL forward pass')

        # 7. Decoder forward pass
        preds = inference_helper.decoder_forward(
            decoder=decoder, x=x, batch=batch, patch_res=patch_res, surf_stats=model.surf_stats,
        )
        del x, batch, patch_res

        # -------------------------------------


        if (sh_var,lh_var) in surf_vars_names:
            task_pred = preds.surf_vars[sh_var][0, 0]
            ref = labels.surf_vars[sh_var][0,0].to(device)
        else:
            task_pred = preds.atmos_vars[sh_var][0, 0]
            ref = labels.atmos_vars[sh_var][0,0].to(device)

        # Paper uses mean absolute error
        loss = torch.mean(torch.abs(task_pred - ref))
        # loss = torch.utils.checkpoint.checkpoint(loss_fn, ref, task_pred, use_reentrant=False)
        loss.backward()

        for name,param in model.named_parameters():
            if cnt == 0:
                grads[name] = torch.square(param.grad.clone().to('cpu'))
            else:
                grads[name] += torch.square(param.grad.clone().to('cpu'))

        cnt += 1
        mae_losses.append(loss.clone().detach().to('cpu').numpy())
        del preds, task_pred, ref
        gc.collect()
        torch.cuda.empty_cache()

    # finished with loop
    task_dir = base_save_dir / lh_var
    task_dir.mkdir(parents=True, exist_ok=True)

    for name,param in grads.items():
        torch.save(param / float(cnt), task_dir / f'{name}.pt')
    np.save(task_dir / f'LOSSES.npy', np.array(mae_losses))